In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
X_train

array([[-1.27565352, -0.56952951, -1.24591035, ..., -0.48238314,
         0.12471531,  0.7381899 ],
       [-0.27391775, -0.72439208, -0.30179191, ..., -0.44090182,
         0.64303644, -0.33535589],
       [-1.28737148, -0.784488  , -1.3085886 , ..., -0.62137652,
        -0.24551406, -0.14987133],
       ...,
       [ 0.30296668,  0.90282058,  0.45297687, ...,  0.93386338,
         0.15433366,  0.1321299 ],
       [-0.19579798, -0.433158  , -0.26015252, ..., -0.96994342,
        -0.22905942, -0.67828664],
       [-0.74564095, -0.09569628, -0.78437056, ..., -0.7790984 ,
         0.59367252, -0.78828329]])

In [7]:
y_train

,diagnosis
341,B
403,B
234,B
498,M
469,B
...,...
373,M
533,M
194,M
246,B


In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [9]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [10]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [11]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [13]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [14]:
learning_rate = 0.1
epochs = 25

In [15]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

In [16]:
# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.2187785804271698
Epoch: 2, Loss: 0.2826385200023651
Epoch: 3, Loss: 0.023537376895546913
Epoch: 4, Loss: 0.1712118238210678
Epoch: 5, Loss: 0.0788118913769722
Epoch: 6, Loss: 0.03551122918725014
Epoch: 7, Loss: 0.08209837973117828
Epoch: 8, Loss: 0.06962870806455612
Epoch: 9, Loss: 0.08576107025146484
Epoch: 10, Loss: 0.09255640208721161
Epoch: 11, Loss: 0.1876443773508072
Epoch: 12, Loss: 0.094968281686306
Epoch: 13, Loss: 0.04653085023164749
Epoch: 14, Loss: 0.013238375075161457
Epoch: 15, Loss: 0.02722652070224285
Epoch: 16, Loss: 0.042537447065114975
Epoch: 17, Loss: 0.10372766107320786
Epoch: 18, Loss: 0.11065872013568878
Epoch: 19, Loss: 0.09447180479764938
Epoch: 20, Loss: 0.011595414951443672
Epoch: 21, Loss: 0.03593531250953674
Epoch: 22, Loss: 0.04003307595849037
Epoch: 23, Loss: 0.03006403148174286
Epoch: 24, Loss: 0.037391599267721176
Epoch: 25, Loss: 0.005880857352167368


In [17]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9193
